In [1]:
import numpy as np
from numpy import std
from numpy import mean
import pandas as pd

from random import randint
from tensorflow import keras 

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

ds = pd.read_csv('TRNcod.xls', delimiter = "\t")

# Shuffle no dataset
ds = ds.sample(frac=1).reset_index(drop=True)

In [ ]:
# print([d for d in ds.columns])

In [2]:
# inadimplentes = pd.DataFrame(list(filter(lambda x: x == 1, ds['IND_BOM_1_2'])))

# Selecionando quem é inadimplente
inadimplentes = ds[ds['IND_BOM_1_2'] == 1]

# Selecionando quem é adimplente
adimplente = ds[ds['IND_BOM_1_2'] == 0]

global treino_ina
global teste_ina
global valid_ina
global treino_adi
global teste_adi
global valid_adi


# Dividindo datasets
treino_ina = inadimplentes[:int(len(inadimplentes)/2)]
teste_ina  = inadimplentes[int(len(inadimplentes)/2):int((len(inadimplentes)*3)/4)]
valid_ina  = inadimplentes[int((len(inadimplentes)*3)/4):]

treino_adi = adimplente[:int(len(adimplente)/2)]
teste_adi  = adimplente[int(len(adimplente)/2):int((len(adimplente)*3)/4)]
valid_adi  = adimplente[int((len(adimplente)*3)/4):]

# Equalizando tamanho de datasets treino e validação dos inadimplentes ao de adimplentes
treino_ina = treino_ina.loc[treino_ina.index.repeat(2)].drop('INDEX', axis=1)
treino_ina["COPIA"] = treino_ina.duplicated()
treino_ina.sort_values(by="COPIA", inplace=True, ignore_index=True)
treino_ina = treino_ina.iloc[ : ( len(treino_adi) - len(treino_ina) ), :  ]
treino_ina.drop(columns=["COPIA"], axis=1)

valid_ina  = valid_ina.loc[valid_ina.index.repeat(2)].drop('INDEX', axis=1)
valid_ina["COPIA"] = valid_ina.duplicated()
valid_ina.sort_values(by="COPIA", inplace=True, ignore_index=True)
valid_ina = valid_ina.iloc[ : ( len(valid_adi) - len(valid_ina) ), :  ]
valid_ina.drop(columns=["COPIA"], axis=1)
# Fim da equalização

print('Tamanhos de inadimplentes: \nTreino: {}\nTeste: {}\nValidação: {}\n'.format(len(treino_ina.values), len(teste_ina.values), len(valid_ina.values)))
print('Tamanhos de adimplentes: \nTreino: {}\nTeste: {}\nValidação: {}\n'.format(len(treino_adi.values), len(teste_adi.values), len(valid_adi.values)))
print('Colunas: {}'.format(len([d for d in ds.columns])))

Tamanhos de inadimplentes: 
Treino: 127549
Teste: 33524
Validação: 63775

Tamanhos de adimplentes: 
Treino: 127549
Teste: 63774
Validação: 63775

Colunas: 246


In [3]:
# https://stackoverflow.com/questions/29294983/how-to-calculate-correlation-between-all-columns-and-remove-highly-correlated-on

# # Create correlation matrix
# corr_matrix = ds.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find features with correlation greater than 0.89
# to_drop = [column for column in upper.columns if any(upper[column] > 0.89)]

# # Drop features 
# # ds.drop(to_drop, axis=1, inplace=True)
# ds.columns


# Colunas mais correlacionadas
# l = []
# for c in ds.columns[:-1]:
#     l.append({c: ds[c].corr(ds['IND_BOM_1_1'])})
#     # if (ds[c].corr(ds['IND_BOM_1_1']) > 0.0):
#         # print( c, ds[c].corr(ds['IND_BOM_1_1']) )
# l.sort(key=lambda x: list(x.values())[0])
# print(l)

In [9]:
# Passos a fazer
# Esquema de experimentação (passo a passo do que vamos testar)
# MLP & Random Forest

In [4]:
# https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/

from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tensorflow import keras

# MLP
model = Sequential()

# Dropout
layer = Dropout(randint(0,100)/100)

# Regularização
# ???

ds_2 = ds
# ds_2 = ds.iloc[ 0:300, : ]

trY = ds_2['IND_BOM_1_2']
trX = ds_2.drop('IND_BOM_1_2', axis=1)
trX = trX.drop('IND_BOM_1_1', axis=1)
trX = trX.drop('INDEX', axis=1)

# Taxa de parada quando não mais evoluir

model.add(Dense(32,input_dim=243, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')


# Parada antecipada caso em 10 epochs ela deixe de melhorar
paradinha = EarlyStopping(monitor='accuracy', mode='max', patience=10)

history = model.fit(trX, trY, epochs=10000, verbose=1, callbacks=[paradinha])




Epoch 1/10000
12163/12163 [==============================] - 8s 631us/step - loss: 0.6016 - accuracy: 0.6719
Epoch 2/10000
12163/12163 [==============================] - 8s 619us/step - loss: 0.5938 - accuracy: 0.6783
Epoch 3/10000
12163/12163 [==============================] - 8s 622us/step - loss: 0.5920 - accuracy: 0.6793
Epoch 4/10000
12163/12163 [==============================] - 8s 618us/step - loss: 0.5906 - accuracy: 0.6807
Epoch 5/10000
12163/12163 [==============================] - 8s 619us/step - loss: 0.5898 - accuracy: 0.6806
Epoch 6/10000
11979/12163 [============================>.] - ETA: 0s - loss: 0.5889 - accuracy: 0.6820

KeyboardInterrupt: 

In [96]:
def mlp_model(neuronios, camadas, dropout, learning_rate, ativacao, otimizador):
    model = Sequential()
    model.add(Dense(neuronios,input_dim=243, activation=ativacao))
    model.add(Dropout(dropout))
    if camadas==2:
        model.add(Dense(1, activation='sigmoid'))
    if otimizador =='adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif otimizador =='SGD':
        opt = keras.optimizers.SGD(learning_rate=learning_rate)
    elif otimizador =='RMSprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif otimizador =='Adadelta':
        opt = keras.optimizers.Adadelta(learning_rate=learning_rate)
        
    
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=opt)
    return model

n_neuronios = [8, 16, 32, 64, 128, 512, 1024]
n_camadas = [1, 2]
n_dropout = [0.4, 0.2, 0.1]
n_learning_rate = [0.1, 0.01, 0.001]
n_ativacao = ['tanh', 'relu']
n_otimizador = ['adam', 'SGD', 'RMSprop','Adadelta']






# ds_2 = ds
ds_2 = ds.iloc[ 0:10000, : ]

trY = ds_2['IND_BOM_1_2']
trX = ds_2.drop('IND_BOM_1_2', axis=1)
trX = trX.drop('IND_BOM_1_1', axis=1)
trX = trX.drop('INDEX', axis=1)


model = mlp_model(32, 2, 0.1, 0.0005,'relu','adam')

paradinha = EarlyStopping(monitor='accuracy', mode='max', patience=10)
history = model.fit(trX, trY, epochs=10000, verbose=1, callbacks=[paradinha])



Epoch 1/10000
313/313 [==============================] - 0s 668us/step - loss: 0.6424 - accuracy: 0.6427
Epoch 2/10000
313/313 [==============================] - 0s 722us/step - loss: 0.6202 - accuracy: 0.6583
Epoch 3/10000
313/313 [==============================] - 0s 719us/step - loss: 0.6131 - accuracy: 0.6657
Epoch 4/10000
313/313 [==============================] - 0s 725us/step - loss: 0.6050 - accuracy: 0.6729
Epoch 5/10000
313/313 [==============================] - 0s 719us/step - loss: 0.6000 - accuracy: 0.6749
Epoch 6/10000
313/313 [==============================] - 0s 770us/step - loss: 0.5966 - accuracy: 0.6797
Epoch 7/10000
313/313 [==============================] - 0s 827us/step - loss: 0.5913 - accuracy: 0.6815
Epoch 8/10000
313/313 [==============================] - 0s 725us/step - loss: 0.5886 - accuracy: 0.6838
Epoch 9/10000
313/313 [==============================] - 0s 658us/step - loss: 0.5840 - accuracy: 0.6915
Epoch 10/10000
313/313 [==============================]

313/313 [==============================] - 0s 805us/step - loss: 0.4469 - accuracy: 0.7919
Epoch 60/10000
313/313 [==============================] - 0s 700us/step - loss: 0.4403 - accuracy: 0.7994
Epoch 61/10000
313/313 [==============================] - 0s 773us/step - loss: 0.4383 - accuracy: 0.7949
Epoch 62/10000
313/313 [==============================] - 0s 738us/step - loss: 0.4367 - accuracy: 0.8010
Epoch 63/10000
313/313 [==============================] - 0s 760us/step - loss: 0.4361 - accuracy: 0.7987
Epoch 64/10000
313/313 [==============================] - 0s 767us/step - loss: 0.4343 - accuracy: 0.7995
Epoch 65/10000
313/313 [==============================] - 0s 687us/step - loss: 0.4314 - accuracy: 0.8010
Epoch 66/10000
313/313 [==============================] - 0s 728us/step - loss: 0.4311 - accuracy: 0.8009
Epoch 67/10000
313/313 [==============================] - 0s 684us/step - loss: 0.4243 - accuracy: 0.8053
Epoch 68/10000
313/313 [==============================] - 0s 

In [12]:
ds_3 = ds.iloc[ 50001:100001, :]
trY = ds_3['IND_BOM_1_2']
trX = ds_3.drop('IND_BOM_1_2', axis=1)
x, y = model.evaluate(trX, trY, verbose=1)

1563/1563 [==============================] - 1s 626us/step - loss: 5.3330 - accuracy: 0.6543


In [ ]:
# Cuidado: usar esses parâmetros muito elevados, ou o default (100 estimadores e profundidade ilimitada) vai travar seu computador

# Parâmetros default:
# n_estimators=100, *,
# criterion="gini",
# max_depth=None,
# min_samples_split=2,
# min_samples_leaf=1,
# min_weight_fraction_leaf=0.,
# max_features="auto",
# max_leaf_nodes=None,
# min_impurity_decrease=0.,
# min_impurity_split=None,
# bootstrap=True,
# oob_score=False,
# n_jobs=None,
# random_state=None,
# verbose=0,
# warm_start=False,
# class_weight=None,
# ccp_alpha=0.0,
# max_samples=None

ds_2 = ds.iloc[ 0:10000, : ]

x_rf = ds_2.drop(['IND_BOM_1_2','IND_BOM_1_1','INDEX'],axis=1)
y_rf = ds_2['IND_BOM_1_2']


random_forest = RandomForestClassifier(n_estimators=10, max_depth=8)
n_scores = cross_val_score(random_forest, x_rf, y_rf, scoring='accuracy', n_jobs=1, error_score='raise')

print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
# Parâmetros default:
# *,loss='deviance', 
# learning_rate=0.1, 
# n_estimators=100,
# subsample=1.0, 
# criterion='friedman_mse', 
# min_samples_split=2,
# min_samples_leaf=1, 
# min_weight_fraction_leaf=0.,
# max_depth=3, 
# min_impurity_decrease=0.,
# min_impurity_split=None, 
# random_state=None, 
# max_features=None, verbose=0,
# max_leaf_nodes=None, 
# warm_start=False,
# validation_fraction=0.1, 
# n_iter_no_change=None, 
# tol=1e-4,
# ccp_alpha=0.0

gradient_boost = GradientBoostingClassifier(n_estimators = 5, max_depth = 8)
n_scores = cross_val_score(gradient_boost, x_rf, y_rf, scoring='accuracy', n_jobs=1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))